In [ ]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pyarabic.araby as araby
import re ,string
import fasttext
from gensim.models.phrases import Phrases, Phraser
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
import numpy
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, Dense, Dropout, Reshape
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import load_model
import nltk
nltk.download('stopwords')
numpy.random.seed(1)
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df=pd.read_csv('/content/transformed.csv', encoding='utf-8-sig')

In [ ]:
shuffled = df.sample(frac=1, random_state=42) 

selected_rows = shuffled.head(65000)

print(selected_rows['dialect'].unique())


['SA' 'KW' 'OM' 'SY' 'PL' 'LB' 'SD' 'EG' 'MA' 'LY' 'IQ' 'AE' 'BH' 'QA'
 'JO' 'YE' 'TN' 'DZ']


In [ ]:
selected_rows['transformed'].unique()

array([ 8, 13, 14,  4,  3, 12, 15, 11,  7,  1,  0, 16, 17,  2,  6,  9,  5,
       10])

In [ ]:
column_name = 'fineText'
has_null_values = selected_rows[column_name].isnull().any()

if has_null_values:
    print(f"The column '{column_name}' contains null values.")
else:
    print(f"The column '{column_name}' does not contain null values.")

The column 'fineText' contains null values.


In [ ]:
selected_rows.dtypes

id              int64
dialect        object
transformed     int64
fineText       object
dtype: object

In [ ]:
selected_rows['fineText'] = selected_rows['fineText'].fillna('').astype(str)

<ipython-input-58-74fcc641c0f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['fineText'] = selected_rows['fineText'].fillna('').astype(str)


In [ ]:
print(selected_rows['fineText'].apply(type).unique())

[<class 'str'>]


# ***Step 1: Preprocessing***

**Remove Arabic Diacritization (tashkeel) like fatha, damma, kasra, shaddah, etc:**

In [ ]:
selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x: araby.strip_tashkeel(str(x)))


<ipython-input-60-45f91e9c501a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x: araby.strip_tashkeel(str(x)))


In [ ]:
selected_rows.head(100)

,id,dialect,transformed,fineText
217576,1161331866680664064,SA,8,نجاح حج هذا العام منتدى الوطن السعودى علم الل...
344491,649669594408349696,KW,13,بنفس هاليوم قبلسنه تم ايقاف سياره فيها اثنين ش...
380119,628467008678400000,OM,14,احسن الظن فيهم يمكن التبن حاجه حلوه عندهم
133458,1018050664906329984,SY,4,هادا من رضا امك عليكى بعتلك ناس حنييتهم بتشبه...
98921,1171931771346259968,PL,3,تامر عاشور اكيد الالبوم الجاى مش حيعمله كئيب
...,...,...,...,...
291489,960587892908228608,EG,11,فى السعيد كمان بس عادى دول بيلعبو فى نادى مصر...
388205,878288196789911552,OM,14,زمان كنا نساعد ابوى فالحلوى
410938,883712080259493888,AE,16,حتى زحل حر لكن عندنا مطر
360858,1161926977651015680,KW,13,اللى دخلوهم مادخلوهم بالخش وكل شى مكشوف ومعرو...


Removing Arabic Stop Words

In [ ]:
stop_words = set(stopwords.words('arabic'))

def process_text(text):
    if isinstance(text, str):
        return ' '.join([word for word in text.split() if word not in stop_words])
    return ''

df['fineText'] = df['fineText'].apply(process_text)


**Remove escape codes like \n, \t, \\, etc from text**

In [ ]:
selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x :re.sub(r"(\\n|\\r|\\t|\\)", "", x).strip())

<ipython-input-63-ad9cd3a211d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x :re.sub(r"(\\n|\\r|\\t|\\)", "", x).strip())


In [ ]:
selected_rows.head(1000)

,id,dialect,transformed,fineText
217576,1161331866680664064,SA,8,نجاح حج هذا العام منتدى الوطن السعودى علم اللى...
344491,649669594408349696,KW,13,بنفس هاليوم قبلسنه تم ايقاف سياره فيها اثنين ش...
380119,628467008678400000,OM,14,احسن الظن فيهم يمكن التبن حاجه حلوه عندهم
133458,1018050664906329984,SY,4,هادا من رضا امك عليكى بعتلك ناس حنييتهم بتشبه ...
98921,1171931771346259968,PL,3,تامر عاشور اكيد الالبوم الجاى مش حيعمله كئيب
...,...,...,...,...
242834,946146404954722432,DZ,10,انتى ما جبتى طاريه ما احب قاعده التعميم
201566,1118352179998412800,SA,8,صح الله لسانك محمد الحيسونى ولسانك يابوسعد ابد...
186679,1089546761910259584,MA,7,بغا يقول العزى انا راه باقى راجل
111631,1105873512756117632,PL,3,سيبكم من التصدى الرهيب انا اللى مش فاهمه المدا...


**Remove URL from text**

In [ ]:
selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x : re.sub(r'http\S+', '', x, flags=re.MULTILINE))

<ipython-input-65-487c4a0eff67>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x : re.sub(r'http\S+', '', x, flags=re.MULTILINE))


In [ ]:
selected_rows.head(1000)

,id,dialect,transformed,fineText
217576,1161331866680664064,SA,8,نجاح حج هذا العام منتدى الوطن السعودى علم اللى...
344491,649669594408349696,KW,13,بنفس هاليوم قبلسنه تم ايقاف سياره فيها اثنين ش...
380119,628467008678400000,OM,14,احسن الظن فيهم يمكن التبن حاجه حلوه عندهم
133458,1018050664906329984,SY,4,هادا من رضا امك عليكى بعتلك ناس حنييتهم بتشبه ...
98921,1171931771346259968,PL,3,تامر عاشور اكيد الالبوم الجاى مش حيعمله كئيب
...,...,...,...,...
242834,946146404954722432,DZ,10,انتى ما جبتى طاريه ما احب قاعده التعميم
201566,1118352179998412800,SA,8,صح الله لسانك محمد الحيسونى ولسانك يابوسعد ابد...
186679,1089546761910259584,MA,7,بغا يقول العزى انا راه باقى راجل
111631,1105873512756117632,PL,3,سيبكم من التصدى الرهيب انا اللى مش فاهمه المدا...


**Remove username “@handle“ from text**

In [ ]:
selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x: re.sub(r'@\w+\s*', '', x))

<ipython-input-67-f7b1b94a161c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['fineText'] = selected_rows['fineText'].apply(lambda x: re.sub(r'@\w+\s*', '', x))


In [ ]:
selected_rows.head(1000)

,id,dialect,transformed,fineText
217576,1161331866680664064,SA,8,نجاح حج هذا العام منتدى الوطن السعودى علم اللى...
344491,649669594408349696,KW,13,بنفس هاليوم قبلسنه تم ايقاف سياره فيها اثنين ش...
380119,628467008678400000,OM,14,احسن الظن فيهم يمكن التبن حاجه حلوه عندهم
133458,1018050664906329984,SY,4,هادا من رضا امك عليكى بعتلك ناس حنييتهم بتشبه ...
98921,1171931771346259968,PL,3,تامر عاشور اكيد الالبوم الجاى مش حيعمله كئيب
...,...,...,...,...
242834,946146404954722432,DZ,10,انتى ما جبتى طاريه ما احب قاعده التعميم
201566,1118352179998412800,SA,8,صح الله لسانك محمد الحيسونى ولسانك يابوسعد ابد...
186679,1089546761910259584,MA,7,بغا يقول العزى انا راه باقى راجل
111631,1105873512756117632,PL,3,سيبكم من التصدى الرهيب انا اللى مش فاهمه المدا...


In [ ]:
print(len(selected_rows))

65000


## TFID

Specifying X and y

In [ ]:
sentences = selected_rows['fineText']
labels = selected_rows['transformed'] 

X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42)


Converting 'dialects' from categorical labels into numeric form

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)
y_train_categorical = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_categorical = to_categorical(y_test_encoded, num_classes=num_classes)

Feature extraction using Tfid & reshaping to fit the LSTM architecture

In [ ]:
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
vectorizer.fit(X_train)

X_train_tfidf = vectorizer.transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

X_train_tfidf = X_train_tfidf.reshape(X_train_tfidf.shape[0], 1, X_train_tfidf.shape[1])
X_test_tfidf = X_test_tfidf.reshape(X_test_tfidf.shape[0], 1, X_test_tfidf.shape[1])


LSTM Architecture

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(X_train_tfidf.shape[1], X_train_tfidf.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

checkpoint = ModelCheckpoint('model_checkpoint.h5', save_weights_only=True, save_best_only=True)

history = model.fit(X_train_tfidf, y_train_categorical, validation_data=(X_test_tfidf, y_test_categorical), epochs=150, batch_size=512, callbacks=[early_stopping, checkpoint])



loss, accuracy = model.evaluate(X_test_tfidf, y_test_categorical)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/150
102/102 [==============================] - 40s 378ms/step - loss: 2.8775 - accuracy: 0.1437 - val_loss: 2.8623 - val_accuracy: 0.1386
Epoch 2/150
102/102 [==============================] - 38s 374ms/step - loss: 2.8390 - accuracy: 0.1378 - val_loss: 2.8162 - val_accuracy: 0.1249
Epoch 3/150
102/102 [==============================] - 36s 357ms/step - loss: 2.7838 - accuracy: 0.1294 - val_loss: 2.7604 - val_accuracy: 0.1232
Epoch 4/150
102/102 [==============================] - 37s 360ms/step - loss: 2.7276 - accuracy: 0.1298 - val_loss: 2.7134 - val_accuracy: 0.1244
Epoch 5/150
102/102 [==============================] - 38s 378ms/step - loss: 2.6814 - accuracy: 0.1325 - val_loss: 2.6748 - val_accuracy: 0.1305
Epoch 6/150
102/102 [==============================] - 38s 375ms/step - loss: 2.6395 - accuracy: 0.1468 - val_loss: 2.6357 - val_accuracy: 0.1536
Epoch 7/150
102/102 [==============================] - 36s 356ms/step - loss: 2.5942 - accuracy: 0.1794 - val_loss: 2.5925 -

Saving the Model

In [ ]:
model.save('model_checkpoint.h5')

Training in again for better accuracy

In [ ]:
loaded_model = load_model('model_checkpoint.h5')
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
loaded_model.fit(X_train_tfidf, y_train_categorical, validation_data=(X_test_tfidf, y_test_categorical), epochs=100, batch_size=512)

loss1, accuracy1 = model.evaluate(X_test_tfidf, y_test_categorical)
print("Test Loss:", loss1)
print("Test Accuracy:", accuracy1)

Epoch 1/100
102/102 [==============================] - 38s 360ms/step - loss: 1.6959 - accuracy: 0.4745 - val_loss: 1.9682 - val_accuracy: 0.3888
Epoch 2/100
102/102 [==============================] - 38s 371ms/step - loss: 1.6882 - accuracy: 0.4753 - val_loss: 1.9686 - val_accuracy: 0.3892
Epoch 3/100
102/102 [==============================] - 38s 372ms/step - loss: 1.6806 - accuracy: 0.4772 - val_loss: 1.9690 - val_accuracy: 0.3898
Epoch 4/100
102/102 [==============================] - 36s 355ms/step - loss: 1.6740 - accuracy: 0.4787 - val_loss: 1.9696 - val_accuracy: 0.3908
Epoch 5/100
102/102 [==============================] - 36s 352ms/step - loss: 1.6674 - accuracy: 0.4801 - val_loss: 1.9704 - val_accuracy: 0.3903
Epoch 6/100
102/102 [==============================] - 36s 354ms/step - loss: 1.6612 - accuracy: 0.4828 - val_loss: 1.9713 - val_accuracy: 0.3908
Epoch 7/100
102/102 [==============================] - 38s 376ms/step - loss: 1.6540 - accuracy: 0.4839 - val_loss: 1.9723 -

KeyboardInterrupt: ignored